In [370]:
import numpy as np
import pandas as pd
import math
import re
from shapely import Polygon

In [371]:
#FILE_NAME = "test.txt"
#FILE_NAME = "test2.txt"
FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False

In [372]:
MODULES = {}
TYPE_FLIPFLOP = "flipflop"
TYPE_BROADCAST = "broadcast"
TYPE_CONJUNCT = "conjuction"
TYPE_BUTTON = "button"

NAME_BROADCASTER = "broadcaster"
NAME_BUTTON = "button"

PULSE_LOW = -1
PULSE_HIGH = 1

STATE_OFF = -2
STATE_ON = 2

State means:
    - flipflop - False means off
    - conjunction - list of connected modules

In [373]:
def get_module_inst(mod_name,mod_type,descendants):
    desc_list = [x.strip() for x in descendants]
    return {"name":mod_name,"type":mod_type,"descend": desc_list,"state_flip":STATE_OFF,"state_conj":{}}


In [374]:
def get_module_inst_button():
    return get_module_inst(NAME_BUTTON,TYPE_BUTTON,[NAME_BROADCASTER])

In [375]:
def buildModulesFromLines(lines):
    
    cur_x = 0
    cur_y = 0
    
    res = []
    
    for line in lines:
        if line[-1]=="\n": line = line[:-1]
        if len(line)>0:
            module_str,downstream = line.split("->")
            
            module_str =  module_str.strip()
             
            
            if module_str[0] == "%":
                mod_type=TYPE_FLIPFLOP
                module_str = module_str[1:]
            elif module_str[0] == "&":
                mod_type = TYPE_CONJUNCT
                module_str = module_str[1:]
            elif module_str.strip()== NAME_BROADCASTER:
                mod_type = TYPE_BROADCAST
            else:
                print ("HOLY SHIT SHOULDN'T HAPPEN!")
            
            module_inst = get_module_inst(module_str,mod_type,downstream.split(","))

    
            MODULES[module_str] = module_inst
    
    # parse all modules to figure out precedecessors of conjunction modules
    
    reverse_module_map = []
    for mod_name, mod_obj in MODULES.items():
        desc_list = mod_obj["descend"]
        for x in desc_list:
            reverse_module_map.append((x,mod_name))
    
    for mod_name, mod_obj in MODULES.items():
        mod_type = mod_obj["type"]
        if mod_type == TYPE_CONJUNCT:
            predec_list = {y:PULSE_LOW for x,y in reverse_module_map if x == mod_name}
            mod_obj["state_conj"] = predec_list
            MODULES[mod_name]=mod_obj
            
    MODULES[NAME_BUTTON]=get_module_inst_button()
    
    return MODULES
    
            

In [376]:
def process_single_pulse(orig_name,dest_name,pulse_type,all_modules):
    if dest_name not in all_modules:
        return []
    obj_dest = all_modules[dest_name]
    obj_dest_type = obj_dest["type"]
    if DIAG2: print("PULSE",orig_name,pulse_type,dest_name,"---",obj_dest_type)
    
    #to_send 
    if obj_dest_type == TYPE_BROADCAST:
        # send same pulse to all
        to_send = pulse_type
    elif obj_dest_type == TYPE_FLIPFLOP:
        if pulse_type == PULSE_HIGH:
            return []
        else:
            obj_dest["state_flip"] = - obj_dest["state_flip"]
            if obj_dest["state_flip"] == STATE_ON:
                # send HIGH PULSE to all
                to_send = PULSE_HIGH
            else:
                to_send = PULSE_LOW
    elif obj_dest_type == TYPE_CONJUNCT:
        obj_dest["state_conj"][orig_name] = pulse_type
        if DIAG: print(">>",obj_dest["state_conj"])
        if all([x==PULSE_HIGH for x in obj_dest["state_conj"].values()]):
            to_send = PULSE_LOW
        else:
               to_send = PULSE_HIGH
        if DIAG: print("<<",obj_dest["state_conj"])
    else:
        print("INCORRECT TARGET MODULE!!!")
        return []
    
    new_pulses = [(dest_name,x,to_send) for x in obj_dest["descend"]]  
    if DIAG: print("New pulses: ",new_pulses)
    return new_pulses

In [377]:
def process_pulses(modules):
    low_count = 0
    high_count = 0 
    pulse_list = [(NAME_BUTTON,NAME_BROADCASTER,PULSE_LOW)]
    while len(pulse_list)>0:
        pulse = pulse_list[0]
        if pulse[2] < 0:
            low_count+=1
        else:
            high_count +=1
        pulse_list = pulse_list[1:]
    
        pulse_list+= process_single_pulse(*pulse,modules)
        if DIAG: print("----\n",pulse_list)
    return (low_count,high_count,modules)

In [378]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    MODULES = buildModulesFromLines(lines)
    
   
    
    
    return MODULES
    

In [379]:
modules = parseFile(FILE_NAME)
print(modules)
PULSES_COUNT = 1000
low_count = 0
high_count = 0
counter=0
while counter < PULSES_COUNT:    
    low_c,high_c,modules=process_pulses(modules)
    low_count+=low_c
    high_count+=high_c
    counter+=1
res = low_count*high_count
print("Result",low_count*high_count)

{'kl': {'name': 'kl', 'type': 'conjuction', 'descend': ['ll'], 'state_flip': -2, 'state_conj': {'ff': -1}}, 'vd': {'name': 'vd', 'type': 'flipflop', 'descend': ['ff', 'mb'], 'state_flip': -2, 'state_conj': {}}, 'dx': {'name': 'dx', 'type': 'flipflop', 'descend': ['hb', 'fx'], 'state_flip': -2, 'state_conj': {}}, 'jj': {'name': 'jj', 'type': 'flipflop', 'descend': ['xt', 'th'], 'state_flip': -2, 'state_conj': {}}, 'ld': {'name': 'ld', 'type': 'flipflop', 'descend': ['fq', 'ff'], 'state_flip': -2, 'state_conj': {}}, 'bn': {'name': 'bn', 'type': 'flipflop', 'descend': ['ff', 'lg'], 'state_flip': -2, 'state_conj': {}}, 'mv': {'name': 'mv', 'type': 'flipflop', 'descend': ['hb', 'mx'], 'state_flip': -2, 'state_conj': {}}, 'mx': {'name': 'mx', 'type': 'flipflop', 'descend': ['xp'], 'state_flip': -2, 'state_conj': {}}, 'qm': {'name': 'qm', 'type': 'flipflop', 'descend': ['gz', 'tj'], 'state_flip': -2, 'state_conj': {}}, 'zd': {'name': 'zd', 'type': 'flipflop', 'descend': ['zp'], 'state_flip': 

In [380]:
#res == 32000000
res == 11687500

False